In [1]:
import parmed
import numpy as np
import itertools

from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

from sys import stdout

from scipy.spatial.distance import cdist


you got numpy


# Load inputs:

Loading the protein/bilayer system:

Using the amber input files here because it simplifies openmm system creation due to charmm-style glyP terminal cap not being recognised. 

In [2]:

prot_pdb = PDBFile('./data/step3_pbcsetup.pdb')
omm_forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml', 'amber14/lipid17.xml')
prot_system = omm_forcefield.createSystem(prot_pdb.topology, rigidWater=False)
prot_structure = parmed.openmm.load_topology(prot_pdb.topology,
                                           prot_system,
                                           xyz=prot_pdb.positions)

protein_indices = np.array([count for count, i in enumerate(prot_pdb.topology.atoms()) if i.residue.name not in ["DPPC", "CL", "NA", "HOH"]])

Load parsley-parameterized drug system:

In [4]:
drug_system = XmlSerializer.deserialize(open('./data/drug_system.xml').read())

drug_pdbfile = PDBFile('./data/drug.pdb')

drug_structure = parmed.openmm.load_topology(drug_pdbfile.topology,
                                                drug_system,
                                                xyz=drug_pdbfile.positions)

# Add drug molecules, randomly placed

Add drug molecules at random orientations and x,y,z (but keeping within the bilayer):

In [5]:
##first, get the appropriate x,y,z ranges:
protein_system_coordinates = prot_structure.coordinates

#how far off from the box do you want to look for new COMs?
offset = 10

x_range = (np.min(protein_system_coordinates[:,0]), np.max(protein_system_coordinates[:,0])+offset)
y_range = (np.min(protein_system_coordinates[:,1]), np.max(protein_system_coordinates[:,1])+offset)

#z_range can be whole box since there's no lipid here. 
z_range = (np.min(protein_system_coordinates[:,2]), np.max(protein_system_coordinates[:,2])+offset)


In [6]:
##generates a rotation matrix using three random angles chosen uniformly in range -np.pi, np.pi
def make_rotation_matrix():
    angles = np.random.uniform(-np.pi, np.pi, (3))
    c1, s1 = np.cos(angles[0]), np.sin(angles[0])
    c2, s2 = np.cos(angles[1]), np.sin(angles[1])
    c3, s3 = np.cos(angles[2]), np.sin(angles[2])
    return np.array(
        [[  c2*c3            , -c2*s3            ,  s2               ],
         [  c1*s3 + c3*s1*s2 ,  c1*c3 - s1*s2*s3 , -c2*s1            ],
         [  s1*s3 - c1*c3*s2 ,  c3*s1 + c1*s2*s3 ,  c1*c2            ]]).T

In [7]:
##how close can the proposed drug molecule be to an existing atom coordinate?
buffer = 3 #this is angstrom here

orig_drug_coords = drug_structure.coordinates

num_drugs = 0
num_wanted = 15

complex_structure = copy.copy(prot_structure)

while num_drugs<num_wanted:
    #make a proposed COM:
    x = np.random.uniform(x_range[0], x_range[1])
    y = np.random.uniform(y_range[0], y_range[1])
    z = np.random.uniform(z_range[0], z_range[1])
    
    #make a random proposed three-axis rotation:
    rot_mat = make_rotation_matrix()
    
    #generate new coordinates for the drug:
    new_coords = np.matmul(orig_drug_coords, rot_mat)+np.array([x,y,z])
    
    #minimum distance to any existing atom:
    mindist = np.min(cdist(new_coords, complex_structure.coordinates))
    
    #if no overlaps below 'buffer' tolerance, then add the ligand.
    if mindist>buffer:            
        drug_structure.coordinates = new_coords
        complex_structure = complex_structure+drug_structure            
        num_drugs+=1
        print(mindist, x,y,z)

3.2417404292844156 12.540098816594721 33.507197704080426 41.88919847200739
3.4543600551881117 40.681386611527294 5.932354083155765 -21.644341154103664
5.058284091971368 1.8516883627711422 22.390554182416565 40.617008774057624
4.82296965166705 -29.152996649864942 -10.554183491309981 42.82216975185894
7.39151666990513 36.38857322799674 37.09991325043592 42.55475674215582
3.7456703530978106 38.19772344308489 41.224895201102726 9.430589065212573
3.350208264208166 14.492547323107821 40.15550856461181 -4.266320646065232
6.426396279881272 40.72679961359963 36.87654590895015 -23.648326876404848
4.613084496507382 -10.707970840309763 37.99259391142745 38.47514067200457
4.574752434631987 -22.7563031572041 41.43653636903329 4.533040499526301
3.0539667673553574 -1.8488043045835596 41.32059053309868 -7.344449891725429
3.595027109945358 17.076631524309015 42.274590786765785 -22.19749675496003
7.47242044990842 42.45485704353725 -31.914301566889193 5.66795500439239
8.652838526380885 42.207241065505904 

set periodic boundary conditions,
then shift so that protein COM is at (0,0,0)

In [8]:
#first, shift so that minimum coordinate is at (0,0,0). This allows to set PBC
complex_structure.coordinates = complex_structure.coordinates-np.min(complex_structure.coordinates, axis=0)
#set PBC with a short buffer:
newbox = (np.max(complex_structure.coordinates,axis=0)+np.array(2))*angstroms
complex_structure.box = (newbox[0], newbox[0], newbox[2], 90, 90, 90)

#now return coordinates so that protein COM is at (0,0,0):
complex_structure.coordinates = complex_structure.coordinates-np.mean(complex_structure.coordinates[protein_indices], axis=0)

In [9]:
##Save a PDB just so you can visualize the output as a sanity check:
complex_structure.save('./data/test.pdb', overwrite=True)

# Simulation setup:

In [10]:
#turn into a System object
complex_system = complex_structure.createSystem(nonbondedMethod=PME,
                                                nonbondedCutoff=0.9*nanometer,
                                               constraints=HBonds,
                                                rigidWater=True)



Collect a set of lists storing the atom indices for each drug residue:

In [11]:
resis = list()
indices = list()
for count, a in enumerate(complex_structure.topology.atoms()):
    if a.residue.name=='UNL':
        resis.append(a.residue.index)
        indices.append(count)
        
drugs = list()
unique_resis=list(set(resis))
for r in unique_resis:
    drugs.append([k for j,k in zip(resis,indices) if j==r])


This cell adds a repulsive bias between each pair of drug molecules. This was inspired by the bias used in the SILCS fragment-screening approach. 

In [12]:
drugForce = CustomNonbondedForce("drugForceMultiplier*exp(-12.5*r)")
drugForce.addGlobalParameter("drugForceMultiplier", 0.01*kilocalories_per_mole/angstroms**2)
for particle in range(complex_system.getNumParticles()):
    drugForce.addParticle()
groups = [i for i in itertools.combinations(drugs,2)]
for group in groups:
    drugForce.addInteractionGroup(group[0], group[1])

print(drugForce.usesPeriodicBoundaryConditions())
drugForce.setCutoffDistance(0.9*nanometer)
drugForce.setNonbondedMethod(2)
complex_system.addForce(drugForce)


False


5

In [13]:
##add a barostat. Not only will this initially fix the big vacuums between cells,
##it will be maintained throughout to keep pressure reasonable. 
barostat = MonteCarloMembraneBarostat(1*bar, 200*bar*nanometer, 310*kelvin, MonteCarloMembraneBarostat.XYIsotropic, MonteCarloMembraneBarostat.ZFree)
complex_system.addForce(barostat)

6

In [14]:
#This is the final force to add during the equil phase. 
#This keeps all the protein atoms restrained. It is periodic, so if the protein
#COM happens to hop across the cell, the restraint centres also move.
#It is added last so it can be removed easily later. 
proteinForce = CustomExternalForce("k1*periodicdistance(x, y, z, x0, y0, z0)^2")
proteinForce.addGlobalParameter("k1", 0.1*kilocalories_per_mole/angstroms**2)
proteinForce.addPerParticleParameter("x0")
proteinForce.addPerParticleParameter("y0")
proteinForce.addPerParticleParameter("z0")
for count, coord in enumerate(complex_structure.positions):
    if count in protein_indices:
        proteinForce.addParticle(int(count), [coord[0], coord[1], coord[2]])        

#for count, coord in enumerate(complex_structure.coordinates[protein_indices]):
#    proteinForce.addParticle(int(count), [coord[0], coord[1], coord[2]])        
print("Does it use periodic conditions?", proteinForce.usesPeriodicBoundaryConditions())
complex_system.addForce(proteinForce)


Does it use periodic conditions? True


7

# Equilibrate:

In [15]:
integrator = LangevinIntegrator(310*kelvin, 1/picosecond, 0.002*picoseconds)
#platform = Platform.getPlatformByName('CPU')
platform = Platform.getPlatformByName('CUDA')
prop = {'CudaPrecision':'single', 'CudaDeviceIndex':'0'}
simulation = Simulation(complex_structure.topology, complex_system, integrator, platform)

In [16]:
simulation.context.setPositions(complex_structure.positions)
#simulation.minimizeEnergy(maxIterations=150)

In [17]:
##PSF files don't like having numbered atom types, because at 10,000 VMD fails
for a in complex_structure.atoms:
    a.type = '0'
complex_structure.save('output.psf', overwrite=True)
complex_structure.save('output.pdb', overwrite=True)


In [18]:
simulation.reporters.append(DCDReporter('output.dcd', 5000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True, speed=True))

In [19]:

one_ns = 500000
for i in range(10):
    simulation.step(one_ns*1)
    simulation.context.setParameter('k1', max(0, float(10-i)))



with open('solvated_equilibrated_system.xml', 'w') as f:
    f.write(
            XmlSerializer.serialize(
                complex_system
            )
    )
    
with open('solvated_equilibrated_state.xml', 'w') as f:
    f.write(
            XmlSerializer.serialize(
                simulation.context.getState(getPositions=True,
                                     getForces=True, getEnergy=True,
                                     enforcePeriodicBox=True)
            )
    )

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
1000,-398783.7353105217,288.10996111775864,0
2000,-393077.7288637308,306.67522556609373,261
3000,-391762.7591112538,309.02182901981496,262
4000,-391516.24439554755,309.6086803467963,261
5000,-392636.2291184324,308.3228568194849,252
6000,-391839.797292087,310.1511835107663,253
7000,-392619.70130956615,311.4572216253451,255
8000,-393044.9939643885,311.4633634400056,255
9000,-392233.5053091005,309.36478402117325,255
10000,-392108.0487570588,309.87158668592434,252
11000,-392075.1466674551,314.2716244573089,252
12000,-393179.43654399225,312.59660527419925,253
13000,-394151.568118816,308.9811454698981,253
14000,-393923.07312079915,312.93720373041054,253
15000,-393744.323051204,312.2611469102288,251
16000,-393700.4178405958,311.8046413518432,252
17000,-395297.0179694288,314.1018914320999,252
18000,-393227.9583675016,308.2837139135063,252
19000,-394681.83313435316,310.6628435396817,253
20000,-395173.0814074117,307.95848081

171000,-410545.3533214631,310.01918319119824,248
172000,-410888.41246353113,312.4810293585014,248
173000,-409787.7325646279,308.2483936904929,248
174000,-409701.62720865733,310.718185908524,248
175000,-410706.2944841909,307.36232861128826,248
176000,-409915.9513258394,306.74237526246355,248
177000,-410951.2798520697,307.9941090564737,248
178000,-410934.0861229345,311.9558481409715,248
179000,-410833.7199859503,309.53557013247547,248
180000,-410061.12381300726,311.6766602677051,247
181000,-409628.0385275441,309.87351073004356,248
182000,-410124.2037922917,309.5559897070619,248
183000,-410963.49883703166,306.2286948895171,248
184000,-410848.67744242307,309.3534144012724,248
185000,-411121.9535830561,310.0629813825607,247
186000,-410119.52846360486,309.6680810833815,247
187000,-410700.8135552369,311.394915316551,247
188000,-410502.66190431034,308.2467386896508,247
189000,-409973.25532180513,304.32064443211857,247
190000,-411361.34429880395,308.04766466691484,247
191000,-411426.4954123271,

339000,-410892.87252377486,310.5977568632826,245
340000,-411676.69203249924,312.0782230439707,245
341000,-410802.49329306325,310.4313057153297,245
342000,-410746.3548411678,312.62311400789923,245
343000,-410293.7751208076,309.70542782236106,245
344000,-410066.7777413633,305.4270194635434,245
345000,-411301.61815304006,310.12301989847236,245
346000,-410328.7500904193,308.4749977832421,245
347000,-410371.5872700589,307.1342273887374,245
348000,-410695.3222291183,309.5539835105217,245
349000,-410044.764978684,311.7204257432547,245
350000,-410098.76699105673,308.6937636357108,245
351000,-410193.6596426957,311.48674353939356,245
352000,-411473.13515042,310.4425079227091,245
353000,-411319.3542141523,307.6921450364461,245
354000,-411635.4282648566,308.59501321576863,245
355000,-411435.59275197494,306.0323790416228,245
356000,-411056.0998421677,310.4546284747391,245
357000,-410861.9509966506,310.38352895212205,245
358000,-410139.27168340445,310.0574887386757,245
359000,-410424.6690979209,309.

507000,-410582.7220030662,308.6326424030669,245
508000,-410336.96969862795,311.16126510576356,245
509000,-411140.8911250348,311.2253092869274,245
510000,-410775.94766148506,310.67393143388705,245
511000,-411006.7980676105,309.84424752537626,245
512000,-409993.61076168343,310.7190446434016,245
513000,-411824.85684904293,313.529801883481,245
514000,-410396.52000224916,311.72055409709907,245
515000,-410438.8932589225,308.6040025913815,245
516000,-410266.61957628326,308.2422442676566,245
517000,-410726.06824846007,309.25280459414006,245
518000,-410804.0437215874,309.89735186900737,245
519000,-410713.30915126763,312.0386044411335,245
520000,-411077.5313821479,311.12657313086055,245
521000,-411270.46552897664,310.8794879467011,245
522000,-411551.2033630735,312.12156186847324,245
523000,-410729.19959353586,311.2605658254978,245
524000,-410838.8292404432,313.11196195011564,245
525000,-411026.49231146974,312.40251665752595,245
526000,-410095.28623616346,306.872077205761,245
527000,-410873.64940

676000,-409949.90503807133,312.56849153989674,245
677000,-409295.8801896826,307.61568284710296,245
678000,-411399.80278971116,310.36216170375786,245
679000,-411180.9179448788,311.7968216919492,245
680000,-410723.86878282554,308.4537210617551,245
681000,-411528.022349786,311.0967334826789,245
682000,-409760.9329235314,310.66475216022945,245
683000,-410207.0016118062,308.7690775783516,245
684000,-410981.3916330915,312.230596312458,245
685000,-410519.6280330084,308.59360918225025,245
686000,-410306.937145076,312.3668497210148,245
687000,-410968.45420786855,313.9597814957967,245
688000,-411500.1801474611,306.88616371124476,245
689000,-411651.5110568197,310.24664164846234,245
690000,-410639.7532497409,310.97696496475623,245
691000,-411430.6240829448,312.85168346326975,245
692000,-409950.0059842102,310.3966132458441,245
693000,-410602.0201024276,311.873154150311,245
694000,-411302.9020943241,307.95778528960807,245
695000,-410239.83362145117,310.2348235867863,245
696000,-410318.9034522213,310

...

4317000,-410317.7056665637,307.4305999226383,244
4318000,-410137.0749189637,310.16215414820266,244
4319000,-411144.5015269418,307.7163847186854,244
4320000,-410639.81542579574,310.4008236374383,244
4321000,-410451.9468197236,308.8419280188101,244
4322000,-409684.8324331753,310.0674512251276,244
4323000,-410690.87658814876,309.2105474769985,244
4324000,-410505.37082997337,309.2291187447672,244
4325000,-411133.2892090338,309.4942233548595,244
4326000,-411978.612598235,307.87046501938914,244
4327000,-411414.5860620013,311.3631260990225,244
4328000,-410595.60277824756,308.2299040361136,244
4329000,-410823.87250557914,310.0319747670833,244
4330000,-410334.5354428331,307.3859072989066,244
4331000,-411804.69519121223,307.6599766895407,244
4332000,-411628.4775218698,312.68286188091054,244
4333000,-411226.58255500766,308.8661140412633,244
4334000,-411064.94703846634,308.216556086341,244
4335000,-410769.657386011,311.21419481001715,244
4336000,-410509.29255252937,307.827707449705,244
4337000,-40

4482000,-411660.59194551734,310.7521565940075,244
4483000,-411389.69990231656,311.1139247164723,244
4484000,-411541.1399122707,308.47707468444963,244
4485000,-410908.06048027496,308.1292337022126,244
4486000,-411155.66377975326,312.0442158216162,244
4487000,-411588.36924562557,309.0017303999503,244
4488000,-410767.28385117743,307.39567051533277,244
4489000,-411877.93618188007,310.88620191245377,244
4490000,-411074.0103385907,309.67441906927996,244
4491000,-410383.3046008481,307.662730554277,244
4492000,-410638.2787948537,314.7774129107264,244
4493000,-410549.92766788974,312.51020493131176,244
4494000,-410982.8163692034,308.1616193538919,244
4495000,-411192.98050352116,307.9153892668368,244
4496000,-410627.9732009012,309.71660247028154,244
4497000,-411264.0474407021,309.57816125775173,244
4498000,-410270.58460696554,306.4057676056504,244
4499000,-410227.781154023,309.4298308036872,244
4500000,-409867.39618058805,310.6141081249062,244
4501000,-411266.30241260957,312.08553608936995,244
45

4648000,-411373.39340803307,311.85196901747116,244
4649000,-411184.00579479523,305.1496497100214,244
4650000,-411210.0211005786,309.63135855059454,244
4651000,-410649.64057131764,308.8713398454015,244
4652000,-410360.9218624381,308.0886066920368,244
4653000,-411666.31525720423,310.99245387369103,244
4654000,-411217.9243270999,311.61742826344397,244
4655000,-411571.6410159543,311.9861148267944,244
4656000,-411491.28573814966,309.7119048932302,244
4657000,-410605.7183539339,309.1854978175859,244
4658000,-410428.64446618455,309.1529581229017,244
4659000,-410595.35675683175,313.3075026621147,244
4660000,-410060.9198248277,310.2902302685122,244
4661000,-410860.4978569348,311.10045198792574,244
4662000,-410656.09728123294,311.06751242084596,244
4663000,-411433.626777292,309.9570409883921,244
4664000,-410569.42046071915,310.3799178409948,244
4665000,-411888.91413169657,312.84930755532787,244
4666000,-411287.6958695075,309.5601898980063,244
4667000,-411823.8201949098,307.0475634956877,244
4668

4814000,-411640.61300930125,307.35870390496854,244
4815000,-411080.6266591414,308.6679386281327,244
4816000,-410546.2126842686,308.8970971163954,244
4817000,-410302.9417572012,309.1021358389052,244
4818000,-411231.6588904741,309.9283566142801,244
4819000,-409585.7512879432,309.96157171076925,244
4820000,-411407.1577418444,311.2929788413916,244
4821000,-411039.0449627936,309.75354918424597,244
4822000,-410604.65887950454,310.7117344501007,244
4823000,-410117.86684257234,311.28262783529686,244
4824000,-410977.45305141597,310.7275828655239,244
4825000,-410344.6679185729,310.74901791692656,244
4826000,-411176.9888220134,311.43982098316974,244
4827000,-410219.989746267,309.16746583875187,244
4828000,-410941.8297132456,307.66273917257485,244
4829000,-410474.8795070937,309.4010645482875,244
4830000,-410837.4386810211,311.23207583718425,244
4831000,-410528.97243263293,310.53886857822846,244
4832000,-411112.3126764095,308.643460574768,244
4833000,-410492.5628078955,311.62176085317077,244
483400

4979000,-411548.9829295813,310.2920760970587,244
4980000,-410158.3013708801,313.1158156013362,244
4981000,-410307.0183466298,309.37472951686874,244
4982000,-410310.53200562694,309.57046941312166,244
4983000,-410839.405457468,309.81146953038956,244
4984000,-411441.6385433269,313.1107541631103,244
4985000,-411813.958792849,306.8232079354418,244
4986000,-411183.119845252,308.71938273053,244
4987000,-411132.4650049284,307.61344447703243,244
4988000,-410798.60070430883,308.9342733307559,244
4989000,-411109.3325247599,310.46453703312403,244
4990000,-410974.55974480486,311.3818978358678,244
4991000,-411816.73389068106,310.5613498361498,244
4992000,-411364.874077786,312.10958566560373,244
4993000,-411137.0879768806,309.9766982401083,244
4994000,-410859.82756174635,311.00598717798766,244
4995000,-410216.63338288805,309.9650619726366,244
4996000,-410468.2114042451,312.05644144648613,244
4997000,-410381.5923123143,309.5222554406823,244
4998000,-411356.60405563144,313.7942277645378,244
4999000,-41